In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import files
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.layers import Dense
from keras.models import Sequential
from tensorflow.keras.optimizers import Nadam, Adam
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from numpy import *
import math
import tensorflow as tf
!pip install shap
import shap
import ipywidgets as widgets

In [ ]:
dataset = pd.read_csv("/content/20,65 new.csv")
X = dataset.drop(["Jsc4", "Voc4", "FF4", "Hysteresis4", "PCE4"], axis = 1)
Y = dataset[["Jsc4", "Voc4", "FF4", "Hysteresis4", "PCE4"]]
scalerx = MinMaxScaler(feature_range = (0,1))
scalery = MinMaxScaler(feature_range = (0,1))
# training the scalar with range of data
scalerx.fit(X)
scalery.fit(Y)
# Applying transformation
scaler_X = scalerx.transform(X)
scaler_Y = scalery.transform(Y)
X_train, X_test, Y_train, Y_test = train_test_split(scaler_X,scaler_Y, test_size = 0.2, random_state = 11) 
def build_model():
    model = Sequential()
    # Input Layer
    model.add(Dense(units = 15, input_shape = [len(X.keys())]))
    # Hidden Layer - I
    model.add(Dense(units = 64, activation= 'hard_sigmoid'))
    # Hidden Layer - II
    model.add(Dense(units = 128, activation= 'linear'))
    # Hidden Layer - III
    model.add(Dense(units = 64, activation= 'hard_sigmoid'))
    # Output Layer
    model.add(Dense(units = 5)) 
    
    # Optimizers 
    # Learning rate 'alpha' = (0.00001 - 0.1) 
    optimizers = Adam(learning_rate= 0.0001)

    model.compile(loss = 'mean_squared_error', optimizer = optimizers, metrics = ['mean_squared_error', 
                                                                                  'mean_absolute_error'])
    return model
model = build_model()
model.summary()
history = model.fit(X_train, Y_train, epochs = 500, batch_size = 5, validation_split = 0.3)
plt.figure(figsize=(8, 4), dpi = 100)
plt.plot(pd.DataFrame(history.history)[['mean_squared_error', 'val_mean_squared_error']])


In [ ]:
predictions = model.predict(X_test)
pd_df = pd.DataFrame(predictions)
predictions_1 = model.predict(X_train)
pd_df = pd.DataFrame(predictions_1)


In [ ]:
r2_score(Y_test, predictions)
mean_squared_error(Y_test, predictions)
mean_absolute_error(Y_test, predictions)

In [ ]:
r2_score(Y_train, predictions_1)
mean_squared_error(Y_train, predictions_1)
mean_absolute_error(Y_train, predictions_1)

In [ ]:
explainer = shap.KernelExplainer(model = model.predict, data = X.head(1677), link = "identity")
X_idx = 705

shap_value_single = explainer.shap_values(X = X.iloc[X_idx:X_idx+1,:], nsamples = 1670)
X.iloc[X_idx:X_idx+1,:]
# Create the list of all labels for the drop down list
list_of_labels = Y.columns.to_list()

# Create a list of tuples so that the index of the label is what is returned
tuple_of_labels = list(zip(list_of_labels, range(len(list_of_labels))))

# Create a widget for the labels and then display the widget
current_label = widgets.Dropdown(options=tuple_of_labels,
                              value=0,
                              description='Select Label:'
                              )

# Display the dropdown list (Note: access index value with 'current_label.value')
current_label

In [ ]:
shap.initjs()
f=X.iloc[X_idx:X_idx+1,:]
print(f)
print(f'Current label Shown: {list_of_labels[current_label.value]}')

a=shap.force_plot(base_value = explainer.expected_value[current_label.value],
                shap_values = shap_value_single[current_label.value],
                features = X.iloc[X_idx:X_idx+1,:],matplotlib=True,show=False
                )

print(f'Original size: {plt.gcf().get_size_inches()}')
w, h= plt.gcf().get_size_inches()
plt.gcf().set_size_inches(h*8, h*1.5)
plt.tight_layout()
print(f'New size: {plt.gcf().get_size_inches()}')

plt.savefig('fig_tes1.png', bbox_inches='tight',dpi=500)

In [ ]:
shap_values = explainer.shap_values(X = X.iloc[0:400,:], nsamples = 1670)
shap.initjs()

print(f'Current Label Shown: {list_of_labels[current_label.value]}\n')
shap.summary_plot(shap_values = shap_values[current_label.value],
                  features = X.iloc[0:400,:],plot_size=(15,15),show=False
                  )

plt.savefig('grafic1.png',dpi=700)